In [ ]:
"""
features, shape, iter, cv, public
01, raw+kmeans+encode, 207, 610, 0.37186, 0.371688
02, raw+kmeans,
03, raw, 111, 805, 0.38474, 0.382972
04, cat+reduce, 236, 1205, 0.370010,
05, raw numeric no norm, 111, 676, 0.385506, 0.382042
06, cat+discrete, 390, 1294, 0.361520
07, raw+reduce, 252, 909, 0.383301
08, raw+ranking, 1387, 0.384005, 0.382578
"""

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xgboost as xgb
import vw_utils as vw
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

selected = np.random.rand(train.shape[0]) < 0.8

In [4]:
# ETL
ID = test.Id.values

categorical = ['T1_V' + str(i) for i in list(range(4, 10)) + [11, 12, 15, 16, 17]] + \
                ['T2_V' + str(i) for i in [3, 5, 11, 12, 13]]

numeric = set(train.columns).difference(categorical + ['Hazard', 'Id'])
numeric = list(numeric)

In [5]:
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()
train_c = train.copy()
train_c[numeric] = SS.fit_transform(train_c[numeric])

/home/albertium/anaconda3/lib/python3.4/site-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


In [6]:
# 13.712
X = {'cat': {'A': categorical},
     'num': {'B': numeric}}

# 13.554
# X = {'cat': {'A': categorical+numeric}}

vw.pd2vw(train_c, X, label='Hazard', out='train.vw')
vw.vwRegress('train.vw', out='linear1', passes=10, l1=1E-6, l2=1E-7)
vw.vwPredict('train.vw', model='linear1', metric=True)

{'flag': 0, 'gini': 0.33718241407489669}

In [92]:
yhat = model.predict(dTest)
pd.DataFrame({'Id': ID, 'Hazard': yhat}).reindex_axis(['Id', 'Hazard'], 1).to_csv('../output/pred08.csv', index=0)

------------------------------------Separator ----------------------------------

In [5]:
# count features

count_features = {}
for cat in categorical:
    count_features[cat] = train[cat].value_counts().reset_index().rename(columns={'index': cat, 0: cat+'_cnt'})

tmp_train = train.copy()
tmp_test = test.copy()
for k, v in count_features.iteritems():
    tmp_train = pd.merge(tmp_train, v, on=k)
    tmp_test = pd.merge(tmp_test, v, on=k)
    

In [4]:
%%time
# kmeans features

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()

kmeans = KMeans(80, random_state=12345, n_init=1)
feat_kmeans_train = kmeans.fit_transform(SS.fit_transform(train[numeric]))
feat_kmeans_test = kmeans.transform(SS.transform(test[numeric]))

print 'variance lost:'
print kmeans.inertia_ / KMeans(1, random_state=12345, n_init=1).fit(train[numeric]).inertia_

feat_kmeans_train = pd.DataFrame(feat_kmeans_train).add_prefix('kmeans_')
feat_kmeans_test = pd.DataFrame(feat_kmeans_test).add_prefix('kmeans_')

variance lost:
0.498810867841
Wall time: 21.1 s


In [53]:
# categorical encoding
from sklearn.preprocessing import StandardScaler

feat_encode_train = train[categorical].copy()
feat_encode_test = test[categorical].copy()

for cat in categorical:
    hot_deck = train.groupby(cat).Hazard.mean().reset_index().rename(columns={'Hazard': 'encode_'+cat})
    feat_encode_train = pd.merge(feat_encode_train, hot_deck, on=cat)
    del feat_encode_train[cat]
    
    feat_encode_test = pd.merge(feat_encode_test, hot_deck, on=cat)
    del feat_encode_test[cat]
    
names = feat_encode_train.columns
ss = StandardScaler()
feat_encode_train[names] = ss.fit_transform(feat_encode_train.values)
feat_encode_test[names] = ss.transform(feat_encode_test.values)

encode_T1_V4     1.011131
encode_T1_V5     1.003089
encode_T1_V6     0.999460
encode_T1_V7     1.025445
encode_T1_V8     0.985815
encode_T1_V9     1.000703
encode_T1_V11    1.001235
encode_T1_V12    0.987670
encode_T1_V15    1.007230
encode_T1_V16    1.005126
encode_T1_V17    1.003132
encode_T2_V3     0.998128
encode_T2_V5     1.001219
encode_T2_V11    1.002282
encode_T2_V12    1.004119
encode_T2_V13    0.994556
dtype: float64

In [81]:
%%time
# reduce
from xgboost import XGBRegressor
import xgboost as xgb

feat_reduce_train = pd.DataFrame()
feat_reduce_test = pd.DataFrame()

for num in numeric:
    dTmp = xgb.DMatrix(train[[num]], label=train.Hazard)
    reg = xgb.train({'max_depth':8, 'min_child_weight':30, 'objective':'reg:linear'}, dTmp, 1)
    
    feat_reduce_train['discrete_'+num] = reg.predict(dTmp, pred_leaf=True)
    feat_reduce_train['discrete_'+num] = feat_reduce_train['discrete_'+num].map(str)
    
    feat_reduce_test['discrete_'+num] = reg.predict(xgb.DMatrix(test[[num]]), pred_leaf=True)
    feat_reduce_test['discrete_'+num] = feat_reduce_test['discrete_'+num].map(str)

Wall time: 6.86 s


In [73]:
# discrete

feat_discrete_train = train[numeric].applymap(str)
feat_discrete_test = test[numeric].applymap(str)

In [88]:
# combine all features
cTrain = train[numeric+categorical]
cTest = test[numeric+categorical]

# cTrain = pd.concat([train[numeric+categorical], feat_reduce_train], axis=1)
# cTest = pd.concat([test[numeric+categorical], feat_reduce_test], axis=1)